In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set your dataset folder path on Google Drive
dataset_path = '/content/drive/My Drive/liver cancer dataset'

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Rescale pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2  # Split data into training and validation
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

valid_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Create ResNet-50 model with transfer learning
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Model checkpoint for saving the best weights
model_checkpoint = ModelCheckpoint('liver_cancer_detection.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min', verbose=1)

# Train the model
history = model.fit(train_generator, validation_data=valid_generator, epochs=15, callbacks=[model_checkpoint])
#show matrix
# Load the best model weights
model.load_weights('liver_cancer_detection.h5')

# Predict labels on the validation data
valid_generator.reset()  # Reset the generator to the beginning
y_pred = model.predict(valid_generator, steps=len(valid_generator), verbose=1)
y_pred = np.round(y_pred)  # Round the predictions to 0 or 1

# Get the true labels
y_true = valid_generator.classes

# Calculate the confusion matrix
confusion = confusion_matrix(y_true, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion)

# You can also print a classification report for additional metrics
classification_rep = classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1'])
print("Classification Report:")
print(classification_rep)


# Save the final model
model.save('liver_cancer_detection_final.h5')


Mounted at /content/drive
Found 69 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
94765736/94765736 [==============================] - 1s 0us/step


Epoch 1/15
3/3 [==============================] - ETA: 0s - loss: 1.0930 - accuracy: 0.4638
Epoch 1: val_loss improved from inf to 0.89205, saving model to liver_cancer_detection.h5
3/3 [==============================] - 46s 19s/step - loss: 1.0930 - accuracy: 0.4638 - val_loss: 0.8921 - val_accuracy: 0.6250
Epoch 2/15
3/3 [==============================] - ETA: 0s - loss: 1.2885 - accuracy: 0.5942
Epoch 2: val_loss did not improve from 0.89205
3/3 [==============================] - 14s 4s/step - loss: 1.2885 - accuracy: 0.5942 - val_loss: 1.2716 - val_accuracy: 0.6250
Epoch 3/15
3/3 [==============================] - ETA: 0s - loss: 0.9139 - accuracy: 0.5942
Epoch 3: val_loss improved from 0.89205 to 0.75386, saving model to liver_cancer_detection.h5
3/3 [==============================] - 16s 8s/step - loss: 0.9139 - accuracy: 0.5942 - val_loss: 0.7539 - val_accuracy: 0.3750
Epoch 4/15
3/3 [==============================] - ETA: 0s - loss: 0.8197 - accuracy: 0.4058
Epoch 4: val_loss d

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = load_model('liver_cancer_detection_final.h5')
# Test with a sample image
test_image_path = '/content/drive/My Drive/image test/n (1).bmp'  # Provide the path to your test image

prediction = predict(test_image_path)

if prediction > 0.5:
    print("The model predicts that this image indicates liver cancer.")
else:
    print("The model predicts that this image does not indicate liver cancer.")

def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(200, 200))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.0  # Rescale pixel values to [0, 1]
    return img
def predict(image_path):
    processed_image = preprocess_image(image_path)
    prediction = model.predict(processed_image)
    return prediction[0][0]  # Get the probability of having liver cancer

1/1 [==============================] - 1s 1s/step
The model predicts that this image does not indicate liver cancer.
